In [5]:
from threading import Thread
import time
import psutil
import os 

In [6]:
def work(id, start, end, result):
    total = 0
    for i in range(start, end):
        total += i
    result.append(total)
    return

In [17]:
START, END = 0, 500000000
result = list()

### 한개의 thread만 사용

In [11]:
start = time.perf_counter()

th1 = Thread(target=work, args=(1, START, END, result))
th1.start()
th1.join()


finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

AFTER  CODE: memory_usage_percent: 32.9%
AFTER  CODE: Current memory KB   :    63.832 KB
Finished in 13.96 second(s)


### 두개의 thread 사용

In [12]:
start = time.perf_counter()

th1 = Thread(target=work, args=(1, START, END//2, result))
th2 = Thread(target=work, args=(2, END//2, END, result))

th1.start()
th2.start()
th1.join()
th2.join()

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

AFTER  CODE: memory_usage_percent: 33.4%
AFTER  CODE: Current memory KB   :    63.824 KB
Finished in 14.7 second(s)


### Using Multi-Processing

In [13]:
from multiprocessing import Process, Queue

In [16]:
start = time.perf_counter()

result = Queue()
th1 = Process(target=work, args=(1, START, END//2, result))
th2 = Process(target=work, args=(2, END//2, END, result))

th1.start()
th2.start()
th1.join()
th2.join()
# AFTER  code
memory_usage_dict = dict(psutil.virtual_memory()._asdict())
memory_usage_percent = memory_usage_dict['percent']
print(f"AFTER  CODE: memory_usage_percent: {memory_usage_percent}%")
# current process RAM usage
pid = os.getpid()
current_process = psutil.Process(pid)
current_process_memory_usage_as_KB = current_process.memory_info()[0] / 2.**20
print(f"AFTER  CODE: Current memory KB   : {current_process_memory_usage_as_KB: 9.3f} KB")
result.put('STOP')
total = 0
while True:
    tmp = result.get()
    if tmp == 'STOP':
        break
    else:
        total += tmp

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

AFTER  CODE: memory_usage_percent: 33.3%
AFTER  CODE: Current memory KB   :    64.770 KB
Finished in 0.08 second(s)


### Test Result 

- Multiprocessing 모듈은 threading 모듈과 구현 방식이 거의 같아서 thread 방식의 코드를 쉽게 이식 가능

- 14.6초(Multi-Thread)에서 0.1초(Multi-Processing)로 많은 시간 단축을 보임

- 단, 프로세스는 각자가 고유한 메모리 영역을 가지기 떄문에 쓰레드에 비하며 메모리 사용이 증가

- 싱글 머신 아키텍쳐 -> 분산 어플리케이션으로 쉽게 전환 가능 

- 각각의 프로세스가 자신만의 메모리 공간을 사용하기 떄문에, 프로세스간 데이터 교환을 위해 multiprocessing.Queue 객체를 새용(Pipe 객체도 사용 가능)

### Conclusion

- 병렬처리를 하는 2가지 작업 - Multi Thread, Multi Processing 

- 쓰레드는 가볍지만 GIL로 인해서 한번에 하나의 쓰레드에서만 작동하여 CPU작업이 적고 I/O 작업이 많은 병렬처리 프로그램에 사용하면 효율 증가

- 프로세스는 각자가 고유한 메모리 영역을 가지기 때문에 더 많은 메모리를 필요로 하지만, 각각 프로세스에서 병렬로 CPU작업 가능, 이를 통해 분산 처리 프로그래밍 구현 가능 

### 추가 MultiProcessing Technics 

- 쓰레드의 동기화 도구들 모두 지원 
    - thread -> multiprocessing: Condition, Event, Lock, RLock, Semaphore